# Importing Necessary Libraries

In [1]:
import numpy as np 
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup #for scrapping data from the web page
print('Libraries imported.')

Libraries imported.


# Scrapping Data from Web

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
webpage = requests.get(url)

In [3]:
soup = BeautifulSoup(webpage.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs: #loop to scrap the table column wise
    i = tr.find_all('td')
    if i:
        rows.append(i)

lst = []
for row in rows: #loop to separate data into cells from rows
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)#creating the dataframe
#merging the Neighborhoods in the same postal area
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

df.shape

(103, 3)

# Getting Coordinate Data from the Web and merge it into the above dataframe

In [5]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/ #downloading the Coordinate data file


--2020-08-05 09:47:47--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.55.161.7
Connecting to cocl.us (cocl.us)|169.55.161.7|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2020-08-05 09:47:47--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.55.161.7|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-05 09:47:48--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-05 09:47:48--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y

In [6]:
df_cord = pd.read_csv('GeoCord.csv') #reading the contents of the downloaded file into a data frame

df['Latitude'] = np.nan
df['Longitude'] = np.nan

# For each postcode in df_assigned, find corresponding coordinates in df_cord and assign it to df_assigned
for index in df.index:
    cord_index = df_cord['Postal Code'] == df.loc[index, 'PostalCode']
    df.at[index, 'Latitude'] = df_cord.loc[cord_index, 'Latitude'].values
    df.at[index, 'Longitude'] = df_cord.loc[cord_index, 'Longitude'].values

# Display the results
df.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Analyzing Data


In [7]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## I chose Etobicoke for Clustering

In [9]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="Coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [10]:
etobicoke_data = df[df['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
4,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999


In [11]:
# create map of Etobicoke using latitude and longitude values
map_etobicoke = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

In [12]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [13]:
etobicoke_data.loc[0, 'Neighborhood']

'New Toronto, Mimico South, Humber Bay Shores'

In [14]:
neighborhood_latitude = etobicoke_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = etobicoke_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of New Toronto, Mimico South, Humber Bay Shores are 43.6056466, -79.50132070000001.


In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2a801758005e361f8ffcd4'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b119977f964a520488023e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_liquor_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d186941735',
         'name': 'Liquor Store',
         'pluralName': 'Liquor Stores',
         'primary': True,
         'shortName': 'Liquor Store'}],
       'id': '4b119977f964a520488023e3',
       'location': {'address': '2762 Lake Shore Blvd W',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'btwn 1st & 2nd St',
        'distance': 408,
        'formattedAddress': ['2762 Lake Shore Blvd W (btwn 1st & 2nd St)',
         'Toronto ON M8V 1H1',
         'Cana

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,LCBO,Liquor Store,43.602281,-79.499302
1,New Toronto Fish & Chips,Restaurant,43.601849,-79.503281
2,Domino's Pizza,Pizza Place,43.601583,-79.500905
3,Delicia Bakery & Pastry,Bakery,43.601403,-79.503012
4,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.602069,-79.499400


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

14 venues were returned by Foursquare.


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neighborhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude']
                                  )

New Toronto, Mimico South, Humber Bay Shores
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West
Islington Avenue, Humber Valley Village
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Northwest, West Humber - Clairville


In [22]:
print(etobicoke_venues.shape)
etobicoke_venues.head()

(74, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
2,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Domino's Pizza,43.601583,-79.500905,Pizza Place
3,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,Popeyes Louisiana Kitchen,43.602069,-79.499400,Fried Chicken Joint


In [23]:
etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",9,9,9,9,9,9
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",7,7,7,7,7,7
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",4,4,4,4,4,4
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",16,16,16,16,16,16
"New Toronto, Mimico South, Humber Bay Shores",14,14,14,14,14,14
"Northwest, West Humber - Clairville",4,4,4,4,4,4
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",1,1,1,1,1,1
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",9,9,9,9,9,9
"The Kingsway, Montgomery Road, Old Mill North",2,2,2,2,2,2


In [24]:
print('There are {} uniques categories.'.format(len(etobicoke_venues['Venue Category'].unique())))

There are 41 uniques categories.


In [25]:
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighborhood'] = etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

,Neighborhood,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Bus Line,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Garden Center,Grocery Store,Gym,Hardware Store,Intersection,Kids Store,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Social Club,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Wings Joint
0,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,"New Toronto, Mimico South, Humber Bay Shores",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
etobicoke_onehot.shape

(74, 42)

In [27]:
etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
etobicoke_grouped

,Neighborhood,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Bus Line,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Discount Store,Drugstore,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Garden Center,Grocery Store,Gym,Hardware Store,Intersection,Kids Store,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Pool,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Social Club,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Wings Joint
0,"Alderwood, Long Branch",0.000000,0.000000,0.00,0.0,0.000000,0.0000,0.00,0.000000,0.000,0.111111,0.0000,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.111111,0.0000,0.000,0.0000,0.000000,0.000000,0.000,0.00,0.000000,0.111111,0.222222,0.111111,0.111111,0.00,0.000000,0.0,0.111111,0.111111,0.0000,0.0000,0.0000,0.0000,0.0000
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.000000,0.00,0.0,0.142857,0.0000,0.00,0.142857,0.000,0.142857,0.0000,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.0000,0.000,0.0000,0.142857,0.000000,0.000,0.00,0.142857,0.142857,0.142857,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000
2,"Kingsview Village, St. Phillips, Martin Grove ...",0.000000,0.000000,0.00,0.0,0.000000,0.0000,0.25,0.000000,0.000,0.000000,0.0000,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000,0.25,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.0,0.250000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000
3,"Mimico NW, The Queensway West, South of Bloor,...",0.000000,0.062500,0.00,0.0,0.000000,0.0625,0.00,0.000000,0.000,0.000000,0.0625,0.0625,0.00,0.062500,0.0625,0.000000,0.00,0.062500,0.062500,0.0625,0.000,0.0625,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.062500,0.000000,0.0625,0.0625,0.0625,0.0625,0.0625
4,"New Toronto, Mimico South, Humber Bay Shores",0.071429,0.071429,0.00,0.0,0.000000,0.0000,0.00,0.071429,0.000,0.142857,0.0000,0.0000,0.00,0.071429,0.0000,0.071429,0.00,0.000000,0.071429,0.0000,0.000,0.0000,0.071429,0.071429,0.000,0.00,0.071429,0.071429,0.071429,0.000000,0.000000,0.00,0.071429,0.0,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000
5,"Northwest, West Humber - Clairville",0.000000,0.000000,0.25,0.0,0.000000,0.0000,0.00,0.000000,0.000,0.000000,0.0000,0.0000,0.25,0.000000,0.0000,0.000000,0.25,0.000000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000
6,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,0.000000,0.00,1.0,0.000000,0.0000,0.00,0.000000,0.000,0.000000,0.0000,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000
7,"South Steeles, Silverstone, Humbergate, Jamest...",0.000000,0.000000,0.00,0.0,0.111111,0.0000,0.00,0.000000,0.000,0.000000,0.0000,0.0000,0.00,0.111111,0.0000,0.111111,0.00,0.222222,0.000000,0.0000,0.000,0.0000,0.111111,0.000000,0.000,0.00,0.000000,0.111111,0.111111,0.000000,0.000000,0.00,0.000000,0.0,0.111111,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000
8,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.000000,0.00,0.0,0.000000,0.0000,0.00,0.000000,0.000,0.000000,0.0000,0.0000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.500000,0.000000,0.00,0.000000,0.5,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000
9,Westmount,0.000000,0.000000,0.00,0.0,0.000000,0.0000,0.00,0.000000,0.125,0.125000,0.0000,0.1250,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.0000,0.125,0.0000,0.000000,0.000000,0.125,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.0,0.125000,0.000000,0.0000

In [28]:
etobicoke_grouped.shape

(10, 42)

In [29]:
num_top_venues = 5

for hood in etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1             Pub  0.11
2             Gym  0.11
3    Skating Rink  0.11
4  Sandwich Place  0.11


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
         venue  freq
0   Beer Store  0.14
1    Pet Store  0.14
2     Pharmacy  0.14
3         Café  0.14
4  Pizza Place  0.14


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq
0                 Park  0.25
1             Bus Line  0.25
2       Sandwich Place  0.25
3          Pizza Place  0.25
4  American Restaurant  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
            venue  freq
0     Wings Joint  0.06
1             Gym  0.06
2  Hardware Store  0.06
3          Bakery  0.06
4     Flower Shop  0.06


----New Toronto, Mimico South, Humber Bay Shores----
                 venue  freq
0          Coffee Shop  0.14
1  Americ

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etobicoke_grouped['Neighborhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Pool,Coffee Shop,Pharmacy,Chinese Restaurant,Flower Shop
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pharmacy,Liquor Store,Beer Store,Café,Coffee Shop,Pet Store,Pizza Place,Wings Joint,Fried Chicken Joint,Flower Shop
2,"Kingsview Village, St. Phillips, Martin Grove ...",Pizza Place,Sandwich Place,Bus Line,Park,Wings Joint,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore
3,"Mimico NW, The Queensway West, South of Bloor,...",Wings Joint,Burger Joint,Grocery Store,Gym,Hardware Store,Thrift / Vintage Store,Kids Store,Fast Food Restaurant,Discount Store,Convenience Store
4,"New Toronto, Mimico South, Humber Bay Shores",Coffee Shop,American Restaurant,Café,Gym,Fast Food Restaurant,Liquor Store,Mexican Restaurant,Pet Store,Pharmacy,Pizza Place


In [32]:
kclusters = 5
etobicoke_grouped_clustering = etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 2, 1, 0, 4, 0], dtype=int32)

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels',kmeans.labels_)

etobicoke_merged = etobicoke_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [34]:
etobicoke_merged.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,0.0,Coffee Shop,American Restaurant,Café,Gym,Fast Food Restaurant,Liquor Store,Mexican Restaurant,Pet Store,Pharmacy,Pizza Place
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,0.0,Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Pool,Coffee Shop,Pharmacy,Chinese Restaurant,Flower Shop
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,4.0,Pool,River,Wings Joint,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,1.0,Baseball Field,Wings Joint,Convenience Store,Grocery Store,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store
4,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,0.0,Wings Joint,Burger Joint,Grocery Store,Gym,Hardware Store,Thrift / Vintage Store,Kids Store,Fast Food Restaurant,Discount Store,Convenience Store
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,0.0,Pharmacy,Liquor Store,Beer Store,Café,Coffee Shop,Pet Store,Pizza Place,Wings Joint,Fried Chicken Joint,Flower Shop
8,M9P,Etobicoke,Westmount,43.696319,-79.532242,0.0,Pizza Place,Intersection,Sandwich Place,Middle Eastern Restaurant,Discount Store,Chinese Restaurant,Coffee Shop,Burger Joint,Bus Line,Café
9,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,3.0,Pizza Place,Sandwich Place,Bus Line,Park,Wings Joint,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore


In [35]:
etobicoke_merged.dropna(axis=0,inplace=True)
etobicoke_merged['Cluster Labels']=etobicoke_merged['Cluster Labels'].astype('int32')


In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etobicoke_merged['Latitude'], etobicoke_merged['Longitude'], etobicoke_merged['Neighborhood'], etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 0, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,0,Coffee Shop,American Restaurant,Café,Gym,Fast Food Restaurant,Liquor Store,Mexican Restaurant,Pet Store,Pharmacy,Pizza Place
1,Etobicoke,0,Pizza Place,Gym,Skating Rink,Sandwich Place,Pub,Pool,Coffee Shop,Pharmacy,Chinese Restaurant,Flower Shop
4,Etobicoke,0,Wings Joint,Burger Joint,Grocery Store,Gym,Hardware Store,Thrift / Vintage Store,Kids Store,Fast Food Restaurant,Discount Store,Convenience Store
7,Etobicoke,0,Pharmacy,Liquor Store,Beer Store,Café,Coffee Shop,Pet Store,Pizza Place,Wings Joint,Fried Chicken Joint,Flower Shop
8,Etobicoke,0,Pizza Place,Intersection,Sandwich Place,Middle Eastern Restaurant,Discount Store,Chinese Restaurant,Coffee Shop,Burger Joint,Bus Line,Café
10,Etobicoke,0,Grocery Store,Pharmacy,Fried Chicken Joint,Liquor Store,Beer Store,Sandwich Place,Fast Food Restaurant,Pizza Place,Wings Joint,Flower Shop


In [38]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 1, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Etobicoke,1,Baseball Field,Wings Joint,Convenience Store,Grocery Store,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store


In [39]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 2, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,2,Bar,Garden Center,Drugstore,Rental Car Location,Wings Joint,Convenience Store,Grocery Store,Fried Chicken Joint,Flower Shop,Fast Food Restaurant


In [40]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 3, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Etobicoke,3,Pizza Place,Sandwich Place,Bus Line,Park,Wings Joint,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore


In [41]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 4, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Etobicoke,4,Pool,River,Wings Joint,Garden Center,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Convenience Store
